## Link Google Drive

In [1]:
#connect google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#access current folder
!ls
import os
os.chdir('/content/drive/My Drive/Project/LDA主题抽取')
!ls
#!git clone https://github.com/AimeeLee77/keyword_extraction

drive  sample_data
2009.xlsx  2014.xlsx  2019.xlsx		 resulet_Reply_2013.csv
2010.xlsx  2015.xlsx  LDA_Topic.ipynb	 resulet_Reply_2015.csv
2011.xlsx  2016.xlsx  Qsubj.csv		 resulet_Reply.csv
2012.xlsx  2017.xlsx  resulet.csv	 stopwords.txt
2013.xlsx  2018.xlsx  resulet_Qsubj.csv  stopWord.txt


## Data Loading

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#读取所有数据
target = 'Reply'#Reply Qsubj
end_year = 2010

def read_data(target, end_year):
  '''
  读取并合并所有列表
  '''
  xlsx_list = []
  for i in range(2009, 2020, 1):
    xlsx_list.append('%d.xlsx'%i)
  data = pd.read_excel('2009.xlsx').astype(str).loc[:,[target]]

  for xlsx_name in xlsx_list[1:]:
    print('deal with %s'%xlsx_name)
    temp = pd.read_excel(xlsx_name).astype(str).loc[:,[target]]
    temp = np.array(temp)
    data = np.vstack([data, temp])
    print('The data number:',len(data))
    if xlsx_name == '%d.xlsx'%end_year :
      break
  data = pd.DataFrame(data, columns=[target])
  print(data.head())
  return data

data = read_data(target, end_year)

deal with 2010.xlsx
The data number: 25812
                                               Reply
0                                                nan
1  我行已与平安人寿签订定向增发协议，请以公司董事会决议公告（2009年第20号）和《深圳发展银...
2                                                nan
3                                         收到您的提议。谢谢！
4  我行已与平安人寿签订定向增发即引进战略投资者的协议，若完成定向增发将会增加公司股本，请以公司...


## Data Processing

In [7]:
import sys,codecs
import jieba.posseg
import jieba.analyse

def dataPrepos(text, stopkey):
  '''
  数据预处理操作：分词，去停用词，词性筛选
  '''
  l = []
  pos = ['n', 'nz', 'v', 'vd', 'vn', 'l', 'a', 'd']  # 定义选取的词性
  seg = jieba.posseg.cut(text)  # 分词
  for i in seg:
      if i.word not in stopkey and i.flag in pos:  # 去停用词 + 词性筛选
          l.append(i.word)
  return l

def data_pp(data, stopkey):
  '''
  对每一行文本预处理
  '''
  idList, abstractList = data.index, data[target]
  corpus = [] # 将所有文档输出到一个list中，一行就是一个文档
  for index in range(len(idList)):
      text = abstractList[index] # 
      text = dataPrepos(text,stopkey) # 文本预处理
      #text = ",".join(text) # 连接成字符串，空格分隔
      corpus.append(text)
  print(corpus[1:3])
  return corpus

stopkey = [w.strip() for w in codecs.open('stopWord.txt', 'r').readlines()]
corpus = data_pp(data, stopkey)

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.955 seconds.
Prefix dict has been built successfully.


[['已', '平安', '人寿', '签订', '定向', '增发', '协议', '请以', '公司', '董事会', '决议', '公告', '股份', '有限公司', '年度', '发行股票', '预案', '事项', '监管', '审批', '请', '关注', '公司', '公告'], []]


##LDA Model

In [10]:
from gensim import corpora,models
import re

def lda_result(corpus):
  # 主题分析
  neg_dict = corpora.Dictionary(corpus)
  neg_corpus = [neg_dict.doc2bow(i) for i in corpus]
  neg_lda = models.LdaModel(neg_corpus,num_topics = 3,id2word = neg_dict)
  neg_theme = neg_lda.show_topics()
  # print(neg_theme)#展示主题
  pattern = re.compile(r'[\u4e00-\u9fa5]+')
  print(pattern.findall(neg_theme[0][1]))# 取出高频词
  pos_key_words =[]
  for i in range(3):
    pos_key_words.append(pattern.findall(neg_theme[i][1]))
  pos_key_words = pd.DataFrame(data=pos_key_words,index=['主题1','主题2','主题3'])
  print(pos_key_words)  
  #记录每段文本的主题
  record_topic=[]
  for i in range(len(neg_corpus)):
    topic = neg_lda.get_document_topics(neg_corpus[i])
    record_topic.append(topic[0][0])
  return record_topic

record_topic = lda_result(corpus)
data['Topic'] = record_topic
print(data.head())
data.to_csv('resulet_%s_%d.csv'%(target,end_year))

['公司', '产品', '市场', '发展', '业务', '技术', '生产', '企业', '行业', '影响']
      0   1    2   3   4    5   6   7   8   9
主题1  公司  产品   市场  发展  业务   技术  生产  企业  行业  影响
主题2  公司  互动   来自  关注  您好  感谢您   易  项目   请  建议
主题3  公司  披露  投资者  经营  您好   业绩  关注  股东  信息  情况
                                               Reply  Topic
0                                                nan      0
1  我行已与平安人寿签订定向增发协议，请以公司董事会决议公告（2009年第20号）和《深圳发展银...      0
2                                                nan      0
3                                         收到您的提议。谢谢！      0
4  我行已与平安人寿签订定向增发即引进战略投资者的协议，若完成定向增发将会增加公司股本，请以公司...      0
